In [34]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder

In [2]:
db = pd.read_csv(r"D:\H_N\Full_HN_DB_deidentified.csv")

C:\Users\johna\anaconda3\envs\phd\lib\site-packages\IPython\core\interactiveshell.py:3186: DtypeWarning: Columns (47,48,49,51,52,55,169,188,205,210,215,235,262,272,278,279,280,281,306) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
db.head()

,ANON_ID,Event Name,Gender,Race,Ethnicity,Marital Status,Complete?,Family History of Cancer,Previous or Other Current Cancer Diagnosis,Type of Cancer (choice=Lung),...,Other Second Cancer Type and Date,New Comorbidity Since Initial Diagnosis or Treatment,New Comorbidity Description and Date,Year of Last Followup,Smoking status at last follow up,Current Status,Disease Status,Other disease status,Year of Death,Cause of Death
0,ANON_1098,First Diagnosis,Male,white,Not Hispanic or Latino,Single,Complete,Yes,No,Unchecked,...,NaN,Yes,Osteoradionecrosis - 2021,2021.0,Never,Alive,NED,NaN,NaN,NaN
1,NaN,Third Diagnosis,NaN,unknown,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ANON_669,First Diagnosis,Male,white,Not Hispanic or Latino,Married,Complete,Yes,No,Unchecked,...,NaN,No,NaN,2012.0,Former,Dead,NaN,NaN,2012.0,This CA or complications of CA
3,NaN,Second Diagnosis,NaN,unknown,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ANON_309,First Diagnosis,Male,white,Not Hispanic or Latino,Married,Complete,Yes,Yes,Unchecked,...,NaN,No,NaN,2015.0,NaN,Alive,NED,NaN,NaN,NaN


In [5]:
care = ["ANON_ID","Gender","Race","Ethnicity","Current Smoking Status (within 1 month of treatment)", "Rounded Age at DX", "T Stage Clinical ", "N stage", "M stage", "HPV status", "Cause of Death"]
sub_db = db[care]

In [8]:
sub_db = sub_db.dropna(subset=["ANON_ID"])

In [10]:
sub_db = sub_db[sub_db['Cause of Death'] != "This CA or complications of CA"]

In [24]:
short_name_map = {"Gender":"gender","Race":"race","Current Smoking Status (within 1 month of treatment)":"smoker","Rounded Age at DX":"age","T Stage Clinical ":"t_stage",
                  "N stage":"n_stage","M stage":"m_stage","HPV status":"hpv"}
one_hot_cols = ['gender','race','smoker','t_stage','n_stage','m_stage','hpv']

In [25]:
renamed_db = pd.DataFrame(index=sub_db.index,columns=short_name_map.values())
for col in sub_db.columns:
    if col in short_name_map.keys():
        renamed_db[short_name_map[col]] = sub_db[col]

In [31]:
renamed_db = renamed_db.fillna('unknown')
for col in renamed_db.columns:
    renamed_db[col] = renamed_db[col].apply(lambda x: x.lower().replace("/",""))

In [43]:
ohe = OneHotEncoder(sparse=False)
data = ohe.fit_transform(renamed_db[one_hot_cols])
headers = ohe.get_feature_names_out()

final_db = pd.DataFrame(index=sub_db.index,columns=headers,data=data)
final_db['age'] = renamed_db['age']
for col in final_db.columns:
    if 'unknown' in col:
        final_db.drop(labels=col,axis=1,inplace=True)
        
final_db.insert(0,'ANON_ID',sub_db['ANON_ID'])

In [45]:
final_db.to_csv(r"D:\H_N\ohe_patient_char.csv",index=False)